<a href="https://colab.research.google.com/github/gulmert89/dsBootcamp/blob/master/odev115_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ödev 11.5 - Derin Öğrenmeye Giriş: Recurrent Neural Network (RNN) Modelleri

1) Derste gösterildiği gibi <code>pip install yfinance</code> komutunu kullanarak, kendinizin karar vereceği 2 farkli hisse senedi seçin ve bu hisse senetlerinin gelecek fiyatlarını LSTM kullanarak tahmin (1-step forecast) edin.

2) RNN'de karşılaşılan sorunları ve neden LSTM'e ihtiyaç duyulduğunu kısaca tartışın.

3) LSTM'de yer alan epoch ve batch size parametrelerinin ne olduğunu araştırın. 1. soruyu farklı epoch ve batch size değerleri ile tekrar çözüp, sonucu rapor edin.

---
# 1. Introduction

## 1.1 Imports

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, os
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras import models, layers